# Протоколы аутентификации

In [1]:
IS_DEBUG = 1

def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

## Протокол PAP
![PAP Protocol](./images/PAP_1.png)

-----

In [2]:
# Использую хэш-функцию ГОСТ 34.11-2012 "Стрибог"
from pygost.gost34112012256 import GOST34112012256

# Эта либа тоже понадобится
import binascii

# Для генерации случайных чисел (КГПСЧ)
from Crypto.Random import get_random_bytes

----

In [3]:
class PAPServer(object):
    def __init__(self):
        self._db = {}
        
    def register_user(self, login: str, password: str):
        trace('[PAP]', f'Attempt to register user with credentials: {login = }, {password = }')
        
        if login in self._db:
            trace('User already exists')
            raise ValueError(f'User {login} is already registered')
            
        #
        # Рассчитаю хэш пароля и запишу в базу
        #
        
        hasher = GOST34112012256()
        hasher.update(password.encode())
        
        self._db[login] = hasher.digest()
        
        trace('[PAP]', f'User successfully registered, password hash: {hasher.hexdigest()}')
        
    def login(self, login: str, password: str) -> bool:
        trace('[PAP]', f'Attempt to login with credentials: {login = }, {password = }')
        
        if login not in self._db:
            trace('[PAP]', "User doesn't exist")
            return False
        
        #
        # Пользователь есть, рассчитаю хэш пароля и сравню с имеющимся
        #
        
        hasher = GOST34112012256()
        hasher.update(password.encode())
        
        real_hash = self._db[login]
        
        trace('[PAP]', f'Password hash: {hasher.hexdigest()}')
        trace('[PAP]', f'Stored hash:   {binascii.hexlify(real_hash).decode()}')
        
        return hasher.digest() == real_hash

In [4]:
class PAPClient(object):
    def __init__(self, login, password):
        self._login  = login
        self._passwd = password
        
        trace('[PAP]', f'User with credentials {self._login}:{self._passwd} created')
        
    def register(self, srv: PAPServer):
        try:
            srv.register_user(self._login, self._passwd)
        except Exception as e:
            trace(e)
    
    def login(self, srv: PAPServer):
        if srv.login(self._login, self._passwd):
            print('[PAP]', f'User {self._login} logged in successfully')
        else:
            print('[PAP]', f'Wrong username or password')

----

In [5]:
# Создаю сервер
server = PAPServer()

In [6]:
# Теперь создаю двух юзеров: 
# - Алису - валидного пользователя
# - Еву - пользователя, пытающегося представиться Алисой
real_alice = PAPClient('Alice', 'Trust_m3_0r_n0t')
eve = PAPClient('Alice', 'try_t0_gu3$$')

[TRACE] [PAP] User with credentials Alice:Trust_m3_0r_n0t created
[TRACE] [PAP] User with credentials Alice:try_t0_gu3$$ created


In [7]:
# Регистрирую валидного пользователя
real_alice.register(server)

[TRACE] [PAP] Attempt to register user with credentials: login = 'Alice', password = 'Trust_m3_0r_n0t'
[TRACE] [PAP] User successfully registered, password hash: bdb147b4b8ae6f408dd51cdb17d6300f3e475b504129ba01e5715899b60366b8


In [8]:
# Захожу под кредами валидного пользователя
real_alice.login(server)

[TRACE] [PAP] Attempt to login with credentials: login = 'Alice', password = 'Trust_m3_0r_n0t'
[TRACE] [PAP] Password hash: bdb147b4b8ae6f408dd51cdb17d6300f3e475b504129ba01e5715899b60366b8
[TRACE] [PAP] Stored hash:   bdb147b4b8ae6f408dd51cdb17d6300f3e475b504129ba01e5715899b60366b8
[PAP] User Alice logged in successfully


In [9]:
# Пытаюсь зайти под неправильными кредами
eve.login(server)

[TRACE] [PAP] Attempt to login with credentials: login = 'Alice', password = 'try_t0_gu3$$'
[TRACE] [PAP] Password hash: 034eda071f984d17f28f9ae214c5d9b9a9fc0e112df98d0e4b5b2eb3d701b314
[TRACE] [PAP] Stored hash:   bdb147b4b8ae6f408dd51cdb17d6300f3e475b504129ba01e5715899b60366b8
[PAP] Wrong username or password


-----

## Протокол CHAP
![CHAP Protocol](./images/CHAP_1.png)

In [10]:
class CHAPServer(object):
    def __init__(self):
        self._db = {}
        
    def register_user(self, login, password):
        trace('[CHAP]', f'Attempt to register user with credentials: {login = }, {password = }')
        
        if login in self._db:
            trace('[CHAP]', 'User already exists')
            raise ValueError(f'User {login} is already registered')
            
        #
        # Запишу в базу
        #
        
        self._db[login] = [password.encode(), None]
        
        trace('[CHAP]', f'User successfully registered, password hash: {hasher.hexdigest()}')
    
    def login(self, login, response):
        trace('[CHAP]', f'Attempt to login with: {login = }, response = {binascii.hexlify(response).decode()}')
        
        if login not in self._db:
            trace('[CHAP]', "User doesn't exist")
            return False
        
        #
        # Пользователь есть, достану челлендж и обнулю
        #
        
        challenge = self._db[login][1]
        self._db[login][1] = None
        
        if challenge is None:
            trace('[CHAP]', f'No challenge was generated for {login = }')
            return False
        
        #
        # Рассчитаю хэш и сравню с респонсом
        #
        
        hasher = GOST34112012256()
        hasher.update(challenge)
        hasher.update(self._db[login][0])
        
        trace('[CHAP]', f'Calculated hash: {hasher.hexdigest()}')
        trace('[CHAP]', f'Received hash:   {binascii.hexlify(response).decode()}')
        
        return hasher.digest() == response
    
    def generate_challenge(self, login):
        trace('[CHAP]', f'Attempt to generate challege for: {login = }')
        
        if login not in self._db:
            trace('[CHAP]', "User doesn't exist")
            return None
        
        challenge = get_random_bytes(16)
        self._db[login][1] = challenge
        
        trace('[CHAP]', f'Generated challenge = {binascii.hexlify(challenge).decode()}')
        
        return challenge

In [11]:
class CHAPClient(object):
    def __init__(self, login, password):
        self._login  = login
        self._passwd = password
        
        trace('[CHAP]', f'User with credentials {self._login}:{self._passwd} created')
        
    def register(self, srv: CHAPServer):
        try:
            srv.register_user(self._login, self._passwd)
        except Exception as e:
            trace(e)
    
    def login(self, srv: CHAPServer):
        challenge = srv.generate_challenge(self._login)
        if challenge is None:
            print('[CHAP]', f'Wrong username or password')
            return
        
        hasher = GOST34112012256()
        hasher.update(challenge)
        hasher.update(self._passwd.encode())
        
        if srv.login(self._login, hasher.digest()):
            print('[CHAP]', f'User {self._login} logged in successfully')
        else:
            print('[CHAP]', f'Wrong username or password')

----

In [12]:
# Создаю сервер
server = CHAPServer()

In [13]:
# Теперь создаю двух юзеров: 
# - Алису - валидного пользователя
# - Еву - пользователя, пытающегося представиться Алисой
real_alice = CHAPClient('Alice', 'Trust_m3_0r_n0t')
eve = CHAPClient('Alice', 'try_t0_gu3$$')

[TRACE] [CHAP] User with credentials Alice:Trust_m3_0r_n0t created
[TRACE] [CHAP] User with credentials Alice:try_t0_gu3$$ created


In [14]:
# Регистрирую валидного пользователя
real_alice.register(server)

[TRACE] [CHAP] Attempt to register user with credentials: login = 'Alice', password = 'Trust_m3_0r_n0t'
[TRACE] name 'hasher' is not defined


In [15]:
# Захожу под кредами валидного пользователя
real_alice.login(server)

[TRACE] [CHAP] Attempt to generate challege for: login = 'Alice'
[TRACE] [CHAP] Generated challenge = c40354f8b4b67fac30e58f86bab2a22e
[TRACE] [CHAP] Attempt to login with: login = 'Alice', response = 3c86ab0f0a6cfdcf95f80563c67feac4518f50bd30c21129b46e44c152bcbe00
[TRACE] [CHAP] Calculated hash: 3c86ab0f0a6cfdcf95f80563c67feac4518f50bd30c21129b46e44c152bcbe00
[TRACE] [CHAP] Received hash:   3c86ab0f0a6cfdcf95f80563c67feac4518f50bd30c21129b46e44c152bcbe00
[CHAP] User Alice logged in successfully


In [16]:
# Пытаюсь зайти под неправильными кредами
eve.login(server)

[TRACE] [CHAP] Attempt to generate challege for: login = 'Alice'
[TRACE] [CHAP] Generated challenge = 4248f79dd04f3d88246bbbdaef38018e
[TRACE] [CHAP] Attempt to login with: login = 'Alice', response = b4db559cc005c2a3bddc75596d7899ac3004056bebca10263f48aa6b62cca2a0
[TRACE] [CHAP] Calculated hash: 0ae9d88d07e249a3c0ca7bf91568d790df4be815e7a143ca6dd08c95adb0ab14
[TRACE] [CHAP] Received hash:   b4db559cc005c2a3bddc75596d7899ac3004056bebca10263f48aa6b62cca2a0
[CHAP] Wrong username or password


----

## Двустронний протокол CHAP

In [17]:
# To be implemented

----

## Протокол S/KEY
![S/KEY Protocol](./images/SKEY_1.png)

In [18]:
# To be implemented